In [1]:
# imports
import os
import json
import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df_csv = pd.read_csv('/Users/zwa216/Documents/BigDataAcademy/DS_Bootcamp/MovieProject/Movie_Analysis.csv')

In [2]:
def change_to_list_df(input_df):
    '''
    because when we load the data from csv, the genre column is string instead of list
    we need to change its type to list
    '''
    def change_to_list(x):
        if type(x) == str:
            return x[1:-1].split(', ')
        else:
            print x

    input_df['genre'] = input_df.apply(lambda x: change_to_list(x['genre']), axis=1)
    return input_df
    
def add_genre(input_df):
    #input_df = combined_df.copy()
    input_df = change_to_list_df(input_df)
    try:
        master_list = []
        for item_list in input_df["genre"]:
            if not item_list is None:
                for item in item_list:
                    master_list.append(item)
    except:
        print 'exception',item_list, item, type(item)
    
    unique_genre_list = list(set(master_list))

    for g in unique_genre_list:
        input_df[g]=0
    try:
        for i in input_df.index:
            if isinstance(input_df.ix[i,'genre'], list):
                for item in input_df.ix[i,'genre']:
                    input_df.ix[i, item]=1
    except:
        print 'd', i, item
    return input_df, unique_genre_list
    
def plot_genre_gross(input_df, unique_genre_list):
    '''
    This function plots the mean domestic gross by different genre
    '''
    dome_gross_genre = dict()
    dome_gross_genre_df = pd.DataFrame(columns=unique_genre_list)
    for item in unique_genre_list:
        dome_gross_genre[item] = input_df.loc[input_df[item]==1]['domestic_gross'].count()
        dome_gross_genre_df[item] = input_df['domestic_gross']*input_df[item]
    #dome_gross_genre = pd.DataFrame(dome_gross_genre, index=[0])
    dome_gross_genre_df=dome_gross_genre_df.replace(0, np.nan)

    # plot boxplot    
    dome_gross_genre_df.boxplot()
    plt.xticks(rotation='vertical')
    plt.ylim([0, 3*10**8])
    plt.show()
    
    # plot count
    plt.bar(range(len(dome_gross_genre)), dome_gross_genre.values())
    plt.xticks(range(len(dome_gross_genre)), dome_gross_genre.keys(), rotation='vertical')
    plt.show()
    return


In [3]:
def add_studios(me_df):
    '''Creates column per each studio if studio is in top 10. The top ten are hard coded.'''

    #List of top ten studios
    top_10_studios = ['Warner Bros. Pictures'
    ,'Universal Pictures'
    ,'Columbia Pictures'
    ,'Paramount Pictures'
    ,'Twentieth Century Fox Film Corporation'
    ,'Sony Pictures Classics'
    ,'Buena Vista Pictures'
    ,'IFC Films'
    ,'Miramax Films'
    ,'Fox Searchlight Pictures']

    def in_top_10(row):
        if row['studio'] in top_10_studios:
            return 1
        else:
            return 0

    for studio in top_10_studios:
        try: 
            me_df[studio] = me_df.apply(in_top_10, axis=1)
        except: 
            print me_df['studio']

    me_df.rename(columns={'Warner Bros. Pictures'                  : 'WarnerBrosPictures'
    ,'Universal Pictures'                     : 'UniversalPictures'
    ,'Columbia Pictures'                      : 'ColumbiaPictures'
    ,'Paramount Pictures'                     : 'ParamountPictures'
    ,'Twentieth Century Fox Film Corporation' : 'TwentiethCenturyFoxFilmCorporation'
    ,'Sony Pictures Classics'                 : 'SonyPicturesClassics'
    ,'Buena Vista Pictures'                   : 'BuenaVistaPictures'
    ,'IFC Films'                              : 'IFCFilms'
    ,'Miramax Films'                          : 'MiramaxFilms'
    ,'Fox Searchlight Pictures'               : 'FoxSearchlightPictures' }, inplace=True)

    return me_df


In [4]:
def create_season(combined_df):
    '''
    This function change release_date to date and create season variables based on released date.
    '''
    combined_df['release_date']=pd.to_datetime(combined_df['release_date'])
    def month_to_season(x):
        if x.month <=4:
            return 'spring'
        elif x.month <=7:
            return 'summer'
        elif x.month <=10:
            return 'fall'
        else: 
            return 'christmas'
    combined_df['release_season'] = combined_df.apply(lambda x: month_to_season(x['release_date']), axis=1)
    
    combined_df.groupby('release_season')['domestic_gross'].mean().plot()
    plt.title('Average domestic gross take')
    plt.show()
    return combined_df


In [5]:
df_csv['genre_len'] = df_csv.apply(lambda x: len(x['genre']), axis=1)

TypeError: ("object of type 'float' has no len()", u'occurred at index 159')

In [7]:
df_studio = add_studios(df_csv)
df_studio.head()

,Unnamed: 0,alt_title,director_x,domestic_gross,mojo_slug,opening_per_theater,opening_weekend_take,production_budget,release_date_limited,release_date_wide,...,WarnerBrosPictures,UniversalPictures,ColumbiaPictures,ParamountPictures,TwentiethCenturyFoxFilmCorporation,SonyPicturesClassics,BuenaVistaPictures,IFCFilms,MiramaxFilms,FoxSearchlightPictures
0,0,102 Dalmatians (2000),NaN,66957026.0,102dalmatians,7353.0,19883351.0,85000000.0,NaN,2000-11-22,...,1,1,1,1,1,1,1,1,1,1
1,1,10 Things I Hate About You (1999),NaN,38178166.0,10thingsihateaboutyou,3668.0,8330681.0,30000000.0,NaN,1999-03-31,...,1,1,1,1,1,1,1,1,1,1
2,2,10 Years (2012),Jamie Linden,203373.0,10years,7569.0,22707.0,NaN,NaN,2012-09-14,...,0,0,0,0,0,0,0,0,0,0
3,3,11 Flowers (2013),NaN,9213.0,11flowers,1758.0,3516.0,NaN,NaN,2013-02-22,...,0,0,0,0,0,0,0,0,0,0
4,4,The 11th Hour (2007),NaN,707343.0,11thhour,15213.0,60853.0,NaN,NaN,2007-08-17,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_studio_genre, unique_list = add_genre(df_studio)
df_studio_genre.head()

nan
nan
nan


,Unnamed: 0,alt_title,director_x,domestic_gross,mojo_slug,opening_per_theater,opening_weekend_take,production_budget,release_date_limited,release_date_wide,...,Mystery,Drama,Action,Documentary,Musical,History,Family,Fantasy,Sport,Biography
0,0,102 Dalmatians (2000),NaN,66957026.0,102dalmatians,7353.0,19883351.0,85000000.0,NaN,2000-11-22,...,0,0,0,0,0,0,1,0,0,0
1,1,10 Things I Hate About You (1999),NaN,38178166.0,10thingsihateaboutyou,3668.0,8330681.0,30000000.0,NaN,1999-03-31,...,0,0,0,0,0,0,0,0,0,0
2,2,10 Years (2012),Jamie Linden,203373.0,10years,7569.0,22707.0,NaN,NaN,2012-09-14,...,0,1,0,0,0,0,0,0,0,0
3,3,11 Flowers (2013),NaN,9213.0,11flowers,1758.0,3516.0,NaN,NaN,2013-02-22,...,0,1,0,0,0,0,0,0,0,0
4,4,The 11th Hour (2007),NaN,707343.0,11thhour,15213.0,60853.0,NaN,NaN,2007-08-17,...,0,0,0,1,0,0,0,0,0,0


In [9]:
df_studio_genre_season = create_season(df_studio_genre)

In [10]:
df_studio_genre_season.head()

,Unnamed: 0,alt_title,director_x,domestic_gross,mojo_slug,opening_per_theater,opening_weekend_take,production_budget,release_date_limited,release_date_wide,...,Drama,Action,Documentary,Musical,History,Family,Fantasy,Sport,Biography,release_season
0,0,102 Dalmatians (2000),NaN,66957026.0,102dalmatians,7353.0,19883351.0,85000000.0,NaN,2000-11-22,...,0,0,0,0,0,1,0,0,0,christmas
1,1,10 Things I Hate About You (1999),NaN,38178166.0,10thingsihateaboutyou,3668.0,8330681.0,30000000.0,NaN,1999-03-31,...,0,0,0,0,0,0,0,0,0,spring
2,2,10 Years (2012),Jamie Linden,203373.0,10years,7569.0,22707.0,NaN,NaN,2012-09-14,...,1,0,0,0,0,0,0,0,0,fall
3,3,11 Flowers (2013),NaN,9213.0,11flowers,1758.0,3516.0,NaN,NaN,2013-02-22,...,1,0,0,0,0,0,0,0,0,spring
4,4,The 11th Hour (2007),NaN,707343.0,11thhour,15213.0,60853.0,NaN,NaN,2007-08-17,...,0,0,1,0,0,0,0,0,0,fall


In [ ]:
df_studio_genre_season.to_csv('/Users/zwa216/Documents/BigDataAcademy/DS_Bootcamp/MovieProject/Movie_Analysis_fnl.csv')